# Space-Track.org boxscore

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np
import glob
import json

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### API config

In [4]:
# username = 'mstiles@grid.news'
# password = 'thespacerace123'
# output = 'json'

In [5]:
!curl -c 'params/cookies.txt' -b 'params/cookies.txt' 'https://www.space-track.org/ajaxauth/login' -d 'identity=mstiles@grid.news&password=thespacerace123'

""

### Boxscore

In [6]:
!curl --limit-rate 100K --cookie 'params/cookies.txt' 'https://www.space-track.org/basicspacedata/query/class/boxscore' -o 'data/raw/boxscore.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35078  100 35078    0     0   106k      0 --:--:-- --:--:-- --:--:--  109k


In [7]:
boxscore = pd.read_json("data/raw/boxscore.json")

In [8]:
boxscore.columns = boxscore.columns.str.lower()

In [9]:
boxscore = boxscore[boxscore["country"] != "ALL"].sort_values(
    "decayed_total_count", ascending=False
)

In [10]:
boxscore.country = (
    boxscore.country.str.title()
    .str.replace("Of", "of", regex=False)
    .str.replace("Peoples", "People's", regex=False)
)

In [11]:
boxscore.iloc[1]

country                      United States of America
spadoc_cd                                          US
orbital_tba                                        16
orbital_payload_count                            4012
orbital_rocket_body_count                         725
orbital_debris_count                             4477
orbital_total_count                              9230
decayed_payload_count                            1370
decayed_rocket_body_count                         726
decayed_debris_count                             4757
decayed_total_count                              6853
country_total                                   16083
Name: 106, dtype: object

In [12]:
boxscore_slim = boxscore[
    [
        "country",
        "spadoc_cd",
        "orbital_tba",
        "orbital_payload_count",
        "orbital_rocket_body_count",
        "orbital_debris_count",
        "orbital_total_count",
    ]
].sort_values("orbital_debris_count", ascending=False)

In [13]:
boxscore_slim["country"] = (
    boxscore_slim["country"]
    .str.replace("Commonwealth of Independent States", "Russia", regex=False)
    .str.replace("People's Republic of China", "China", regex=False)
    .str.replace("To Be Determined/Unknown", "Unknown", regex=False)
    .str.replace(
        "International Telecommunications Satellite Organization (Intelsat)",
        "Intelsat",
        regex=False,
    )
    .str.replace(
        "Orbital Telecommunications Satellite (Globalstar)",
        "Orbital Telecommunications Satellite",
        regex=False,
    )
    .str.replace(
        "European Telecommunications Satellite Organization (Eutelsat)",
        "European Telecommunications Satellite Organization",
        regex=False,
    )
    .str.replace(
        "United States of America",
        "United States",
        regex=False,
    )
)

In [14]:
boxscore_slim.head(10)

,country,spadoc_cd,orbital_tba,orbital_payload_count,orbital_rocket_body_count,orbital_debris_count,orbital_total_count
18,Russia,CIS,5,1552,1048,6043,8648
106,United States,US,16,4012,725,4477,9230
77,China,PRC,82,517,195,3656,4450
32,France,FR,0,80,165,355,600
40,India,IND,0,103,40,75,218
49,Japan,JPN,6,205,62,54,327
25,European Space Agency,ESA,0,96,7,53,156
16,China/Brazil,CHBZ,0,4,0,37,41
72,Orbital Telecommunications Satellite,ORB,0,58,0,17,75
28,European Organization For The Exploitation of Meteorological Satellites,EUME,0,9,0,8,17


In [15]:
boxscore_graphic = (
    boxscore_slim[["country", "orbital_total_count"]]
    .sort_values("orbital_total_count", ascending=False)
    .reset_index(drop=True)
)

In [16]:
boxscore_graphic = boxscore_graphic[boxscore_graphic["country"] != "Unknown"]

In [17]:
boxscore_graphic.head(10)

,country,orbital_total_count
0,United States,9230
1,Russia,8648
2,China,4450
3,France,600
4,United Kingdom,449
5,Japan,327
6,India,218
7,European Space Agency,156
9,Intelsat,92
10,Globalstar,85


In [18]:
boxscore_graphic["orbital_total_count"].sum()

25371

### Export

In [19]:
boxscore_graphic.head(10).to_csv("data/processed/boxscore_graphic.csv", index=False)